In [ ]:
import tensorflow as tf
import numpy as np
import csv
import pandas as pd
import numpy as np
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt

In [ ]:
# Parameters
RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)

days_history = 4
features = 4
learningrate = 0.02
epochs = 100
percent_training = 0.8
mid_layer_size = 32


In [ ]:
# Get dataset

def get_dataset(name):
    df = pd.read_csv(name, sep=';', index_col=0, encoding="Latin1")
    df = df[::-1]
    df.columns = ['Siste', 'Kjoper', 'Selger', 'Hoy', 'Lav', 'Totalt_omsatt_NOK', 'Totalt_antall_aksjer_omsatt', 'Antall_off_handler', 'Antall_handler_totalt', 'VWAP']
    return df


def get_currency_data(name):
    df = pd.read_csv(name, sep=';', index_col=0)
    return df


def append_currency_to_stock_price_df(df_stock, df_currency):
    df_stock["Dollarkurs"] = df_currency["1 USD"]


MHG = get_dataset("data\\MHG_data.csv")
nok = get_currency_data("data\\Valutakurser.csv")

append_currency_to_stock_price_df(MHG, nok)

In [ ]:
# Initialize labels

Up = MHG.Siste.diff(1) > 0

n = len(Up)

train = np.zeros((n, 2))
train[:,0] = (Up.values == 0)
train[:,1] = (Up.values == 1)

cutoff = int(np.floor(n*percent_training))

Y_train = train[:cutoff,:]
Y_test = train[cutoff:,:]

Y_train = Y_train[days_history:]
print(Y_train.shape)
print(Y_test.shape)

In [ ]:
# Initialize training data
input_dimension = features*days_history

Siste_all = np.zeros((n, days_history))
Siste = MHG.Siste.values

Omsatte_all = np.zeros((n, days_history))
Omsatte = MHG.Totalt_antall_aksjer_omsatt.values

Antall_handler_all = np.zeros((n, days_history))
Antall_handler = MHG.Antall_handler_totalt.values

Valutakurs_all = np.zeros((n, days_history))
Valutakurs = MHG.Dollarkurs.values


def normalize(vector, ref):
    ref_min = min(ref)
    ref_max = max(ref)
    vector -= ref_min
    return vector / (ref_max-ref_min)
    

for i in range(days_history):
    Siste_all[:,i] = np.roll(Siste, i)
    Omsatte_all[:,i] = np.roll(Omsatte, i)
    Antall_handler_all[:,i] = np.roll(Antall_handler, i)
    Valutakurs_all[:,i] = np.roll(Valutakurs, i)
   
Siste_all = normalize(Siste_all, Siste)    
Omsatte_all = normalize(Omsatte_all, Omsatte)
Antall_handler_all = normalize(Antall_handler_all, Antall_handler)
Valutakurs_all = normalize(Valutakurs_all, Valutakurs)

plt.plot(Siste_all[cutoff:])
plt.show()


X_data = Siste_all

X_data = np.append(X_data,Omsatte_all, 1)
X_data = np.append(X_data,Antall_handler_all, 1)
X_data= np.append(X_data,Valutakurs_all, 1)


plt.plot(X_data[days_history:])
plt.show()
print(X_data.shape)

X_train = X_data[:cutoff,:]
X_test = X_data[cutoff:,:]
X_train = X_train[days_history:]
print(X_train.shape)
print(X_test.shape)

In [ ]:
# Setup graph
def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=1)
    return tf.Variable(weights)

# Layer's sizes
x_size = X_train.shape[1]  
h_size = mid_layer_size              
y_size = Y_train.shape[1] 

# Symbols
X = tf.placeholder("float", shape=[None, x_size])
y = tf.placeholder("float", shape=[None, y_size])

# Weight initializations
w_1 = init_weights((x_size, h_size))
w_2 = init_weights((h_size, y_size))

# Forward propagation
h    = tf.nn.tanh(tf.matmul(X, w_1)) 
yhat = tf.nn.softmax(tf.matmul(h, w_2))
predict = tf.argmax(yhat, axis=1)

# Backward propagation
cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
updates = tf.train.GradientDescentOptimizer(learningrate).minimize(cost)


In [ ]:
# Run graph
config = tf.ConfigProto(device_count = {'GPU': 0})
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
sess.run(init)

for epoch in range(epochs):
    for i in range(len(X_train)):
        sess.run(updates, feed_dict={X: X_train[i: i + 1], y: Y_train[i: i + 1]})

    train_accuracy = np.mean(np.argmax(Y_train, axis=1) ==
                 sess.run(predict, feed_dict={X: X_train, y: Y_train}))
    test_accuracy  = np.mean(np.argmax(Y_test, axis=1) ==
                 sess.run(predict, feed_dict={X: X_test, y: Y_test}))
    number_of_negatives = np.average( sess.run(predict, feed_dict={X: X_train, y: Y_train}) == 0)
    number_of_positives = np.average( sess.run(predict, feed_dict={X: X_train, y: Y_train}) == 1)
    print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%. Positives: %.2f%%. Negatives %.2f%%."
          % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy, 100. * number_of_positives, 100. * number_of_negatives))


In [ ]:
test_results = (np.argmax(Y_test, axis=1) == sess.run(predict, feed_dict={X: X_test, y: Y_test}))
#print(sess.run(yhat, feed_dict={X: X_test, y: Y_test}))
print(sess.run(predict, feed_dict={X: X_test, y: Y_test}))
print(np.argmax(Y_test, axis=1))




In [ ]:
sess.close()